<a href="https://colab.research.google.com/github/jaroorhmodi/transformer-from-scratch/blob/main/transformer_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Building a Transformer from Scratch

Using the famous paper [***Attention Is All You Need***](https://arxiv.org/pdf/1706.03762.pdf), we will build a transformer model from scratch and train it on some synthetic data.

Following [*The Annotated Transformer*](http://nlp.seas.harvard.edu/annotated-transformer/#training-data-and-batching) along with this [Towards Data Science post](https://towardsdatascience.com/all-you-need-to-know-about-attention-and-transformers-in-depth-understanding-part-1-552f0b41d021#8607) loosely to clarify any confusions when reading the paper.

I will **NOT** be training the model on the same amount of data as the original paper does. It's about 3.5 days of training on 8 GPUs, something I cannot do at the moment. Perhaps I will attempt bigger models in the future! :)

##Preliminary Work

In [123]:
# !pip install -q torchdata==0.3.0 torchtext==0.12 altair GPUtil

In [124]:
import torch
from torch import nn
import math
import copy
import time
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset

RUN_EXAMPLES = True

#I will only ever run this on GPU
#but I want to make this device agnostic.

#FUTURE GOALS: Try to do multi-gpu training in colab in the
#future with a newer, more ambitious paper to duplicate
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [125]:
# Some convenience helper functions used throughout the notebook


def is_interactive_notebook():
    return __name__ == "__main__"


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None

###PAPER COMMENTS:
**Section 1** of the paper basically breaks down part of the motivation behind including attention mechanisms in NLP models. I am familiar with how LSTM and other RNN architectures work and it seems like the impetus (at the time) was to capture inter-word dependencies in a way that is not inherently sequential. This allows for training efficiency to improve by leveraging parallel computation (*which I am funnily enough eschewing in this exercise I am undertaking*). This parallelization effectively increases the effectiveness of the model by allowing for training on larger sets of data.

**Section 2** of the paper breaks down how attention compares to existing approaches employing convolution and sequence-aligned RNNS ([which I think refers to Seq2Seq models](https://towardsdatascience.com/how-to-implement-seq2seq-lstm-model-in-keras-shortcutnlp-6f355f3e5639)).

Attention has an advantage in learning relations between distant word positions (constant number of operations) but it loses some resolution. The Transformer employs **Multi-Head-Attention** to counteract this. This is described later in the paper.

##Model Architecture

In the paper, model architecture is covered in **Section 3**.

The transformer model is similar to prior popular models like *LSTM* in that it has an **encoder-decoder** structure. Encoders map input sequences (symbol representation) to continuous representations which the decoder then maps to symbols. this happens one element at a time and is auto-regressive; which is to say that all previously generated symbols become inputs as the next output symbol is generated.

For $\mathbf{x} = (x_1,…, x_n)\in \mathbf{W}$, $\mathbf{y} = (y_1,…, y_m)\in \mathbf{W'}$, $\mathbf{z} = (z_1,…, z_n)\in \mathbb{R}^n$, the mapping looks something like this:
$$\mathbf{x}→\mathbf{z}→\mathbf{y}$$
Above we have $\mathbf{W}$ and $\mathbf{W'}$ to represent word vocabularies which may or may not be in the same language (different languages might be used in a machine translation task like the original transformer paper used). Although even in the case of the machine translation use case, vocabularies are merged for both input and output since the weight vector of the embedding is shared for input and output embeddings.

![Encoder-Decoder Architecture](https://machinelearningmastery.com/wp-content/uploads/2021/08/attention_research_1-768x1082.png)

In [126]:
class EncoderDecoder(nn.Module):
  """
  This is a skeleton for the encoder-decoder structuer mentioned above.

  I am following the code organization in the Annotated Transformer article mentioned above.
  """
  def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
    super(EncoderDecoder, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.tgt_embed = tgt_embed
    self.generator = generator

  def encode(self, src, src_mask):
    """
    Forward pass of the encoder. Embed the src text
    and mask it. Then you run a fwd pass of the encoder on it.

    This is the shell of the first half of the transformer's
    forward pass. We will code the specfics of this when we
    design the encoder.
    """
    return self.encoder(self.src_embed(src), src_mask)

  def decode(self, memory, src_mask, tgt, tgt_mask):
    """
    This is the shell of the second half of the forward pass of
    the transformer. We will also specify this further ahead.

    The decoder has a memory component because it also takes in
    the already generated words in its output.
    """
    return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

  def forward(self, src, tgt, src_mask, tgt_mask):
    "Take in and process masked src and target sequences."
    return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

***The Annotated Transformer*** also defines the generator portion of the transformer here. This is given in the ***Attention is All You Need*** paper by the linear portion following the decoder along with the softmax step following it. This linear layer is not the same as the linear layers that are in both the encoder and decoder.

In [127]:
class Generator(nn.Module):
  """
  As described above, the Generator follows the Decoder.
  This portion seems relatively simple.
  It's simply a fully connected Linear step followed by softmax.
  """
  def __init__(self, d_model, vocab):
    super(Generator, self).__init__()
    self.proj = nn.Linear(d_model, vocab)

  def forward(self, x):
    return nn.functional.log_softmax(self.proj(x), dim=1)

**Section 3.1**: The paper describes the nature of the Encoder and Decoder here.

###Encoder
    
The encoder has $N$ stacked layers that are all the same. Each layer has a Multi-Head Attention sublayer that feeds into a position-wise fully connected feed-forward network.The output of each sublayer is given by $\text{LayerNorm}(x + \text{Sublayer}(x))$ where $\text{Sublayer}(x)$ is the forward pass function of the sublayer in question.

`LayerNorm` is layer normalization. **The Annotated Transformer** implements it manually so I will do the same to avoid issues further ahead but I believe what is being done here is very similar to the implementation in [`torch.nn.LayerNorm`](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html).

There is also the question of a residual connection between the sublayers alongside the norm. In the paper it is mentioned that a sublayer dropout is implemented before the normalization with $P_{\text{drop}}=0.1$ in **Section 5.4**.

In the paper the encoder has $N=6$ stacked layers.

In [128]:
class LayerNorm(nn.Module):
  def __init__(self, features, eps=1e-6):
    super(LayerNorm, self).__init__()
    self.a_2 = nn.Parameter(torch.ones(features))
    self.b_2 = nn.Parameter(torch.zeros(features))
    self.eps = eps

  def forward(self, x):
    mean = x.mean(-1, keepdim=True)
    std = x.std(-1, keepdim=True)
    return self.a_2*(x-mean)/(std+self.eps) +self.b_2

In [129]:
class SublayerConnection(nn.Module):
  """
  Residual Connection with dropout followed by LayerNorm.

  This accomplishes the aforementioned residual connection
  with normalization used between the sublayers of the Encoder.

  dropout has default prob=0.1 like in the paper.
  """
  def __init__(self, size, dropout=0.1):
    super(SublayerConnection, self).__init__()
    self.norm = LayerNorm(size)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, sublayer):
    return x+self.dropout(sublayer(self.norm(x)))


I think I see why this is self-implemented. This will do LayerNorm for multiple stacked layers as a single layer in an `EncoderLayer` implementation.

In [130]:
def clones(module, N):
  #produces N-stack of identical layers
  return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class Encoder(nn.Module):
  def __init__(self, layer, N):
    super(Encoder, self).__init__()
    self.layers = clones(layer, N)
    self.norm = LayerNorm(layer.size)

  def forward(self, x, mask):
    "Forward pass of the encoder. Include mask."
    for layer in self.layers:
      x = layer(x, mask)
    return self.norm(x)

In [131]:
class EncoderLayer(nn.Module):
  """
  This is the fundamental architecture of a single Encoder Layer.

  The encoder will be made up of N stacked copies of this (N=6).

  We have yet to implement the key feature: Multi-Head Attention.

  This will be implemented and explained below.
  """
  def __init__(self, size, self_attn, feed_forward, dropout=0.1):
    super(EncoderLayer, self).__init__()
    self.self_attn = self_attn #multi-head attention layer
    self.feed_forward = feed_forward
    # N=2 sublayer connections here are not to be thought of
    # as "stacked" but rather they are just the sublayer connection
    # following the MHA layer and then the feedforward layer.
    self.sublayer = clones(SublayerConnection(size, dropout), 2)
    self.size = size

  def forward(self, x, mask):
    x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
    return self.sublayer[1](x, self.feed_forward)


###Decoder
    
The decoder is similar to the encoder in that it employs $N=6$ stacked layers that are exactly alike. It also shares a similar sublayer connection to the Encoder but it has *two* MHA sublayers.

The first performs masked MHA over the output embedding (the mask is there to prevent signal from later words in the output from reaching prediction on earlier words).

The second sublayer performs MHA over the output from the encoder and maintains the sublayer scheme from before.

This all goes into the third sublayer (feed-forward) in a similar fashion to the encoder layer.

In [132]:
class Decoder(nn.Module):

  def __init__(self, layer, N):
    super(Decoder, self).__init__()
    self.layers = clones(layer, N)
    self.norm = LayerNorm(layer.size)

  def forward(self, x, memory, src_mask, tgt_mask):
    for layer in self.layers:
      x = layer(x, memory, src_mask, tgt_mask)
      return self.norm(x)

We will make the `DecoderLayer` class using the `SublayerConnection` class we made before. MHA is yet to be implemented, it will be accounted for in the `DecoderLayer` class.

In [133]:
class DecoderLayer(nn.Module):

  def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
    super(DecoderLayer, self).__init__()
    self.size = size
    self.self_attn = self_attn
    self.src_attn = src_attn
    self.feed_forward = feed_forward
    self.sublayer = clones(SublayerConnection(size, dropout), 3)

  def forward(self, x, memory, src_mask, tgt_mask):
    m = memory
    x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
    x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
    return self.sublayer[2](x, self.feed_forward)

We need to include the mask to mask out subsequent words in the output. This way we avoid the signal from later words (or positions) in the output from reaching earlier ones.

In [134]:
def subsequent_mask(size):
  attn_shape = (1,size,size)
  subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(torch.uint8)
  return subsequent_mask == 0

###Attention

In the paper there are two parts to *Multi-Headed Attention*. There is **Scaled Dot-Product Attention** which feeds into **Multi-Headed Attention**.

![](https://machinelearningmastery.com/wp-content/uploads/2022/03/dotproduct_1.png)

The reason we scale dot product attention by $\frac{1}{\sqrt{d_k}}$ is because as $d_k$ (the dimension of the query and key vectors) goes up, the dot products grow large in magnitude. This makes `softmax` have very small gradients which adversely affects model performance. The scaling of dot products counteracts this precise issue.

Imagine the components of $Q$, $K$ are random variables with mean=0 and variance=1. This means their dot-product has mean=0 and variance=$d_k$=$d_q$.

In [135]:
#First we implement Scaled Dot Product Attention
def attention(Q:torch.Tensor, K:torch.Tensor, V:torch.Tensor, mask=None, dropout=None):
  d_k = Q.size(-1) #d_k is the dimension of queries and keys
  #The reason we divide by d_k is given above.
  scores = torch.matmul(Q, K.transpose(-2,-1)) / math.sqrt(d_k)
  if mask is not None:
    #we fill zeroes with something like -inf
    #this is to bring them close to 0 in softmax
    scores = scores.masked_fill(mask == 0, -1e9)

  p_attn = scores.softmax(dim=-1)
  if dropout is not None:
    p_attn = dropout(p_attn)
  return torch.matmul(p_attn, V), p_attn


**Section 3.2.2**:

We will implement Multi-Headed Attention here. This is the crux of the Transformer model.

I will comment anything I find helpful in explaining the structure in the code itself. This is because even though the general intuition behind attention makes sense, the whole point of this exercise is to understand the specifics of its implementation in the Transformer model.

The "*multi-head*" part of MHA is accomplished by a series of $h$ learned projections where $h$ is the number of heads. We have $h=8$ parallel attention layers. Each of these project $Q$, $K$, $V$ into $d_k$, $d_k$, and $d_v$ dimensions via learned parameter matrices described below.

$$i \in \{1,…,h\}$$
$$W_i^Q, W_i^K \in \mathbb{R}^{d_{\text{model}}×d_k}$$
$$W_i^V \in \mathbb{R}^{d_{\text{model}}×d_v}$$
$$W^O \in \mathbb{R}^{hd_{v}×d_{\text{model}}}$$

Each of the projections for a head are given:
$$QW_i^Q, KW_i^K, VW_i^V$$

And the attention function of each head is modeled:
$$\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$$

The way we bring these heads' output together is by concatenating and mapping back into the model space $\mathbb{R}^{d_{\text{model}}}$ with the parameter matrix $W^O$.

So the Multi-Head Attention function is given:
$$\text{MultiHead}(Q,K,V) = \text{Concat}(\text{head}_1,…,\text{head}_h)W^O$$

In [136]:
#Implementing Multi-Head Attention
class MultiHeadAttention(nn.Module):
  def __init__(self, h, d_model, dropout=0.1):
    super(MultiHeadAttention, self).__init__()
    #make sure model dimension is divisible into h heads
    assert d_model % h == 0
    #Remember the note above, in this paper:
    #𝑑_𝑞=𝑑_𝑘=𝑑_𝑣=𝑑_model/ℎ
    self.d_k = d_model//h #this will act as d_q ,d_v for us
    self.h = h
    #TODO: what are the cloned linears for? FOR PROJECTION
    #TODO: Why are there FOUR and not THREE? LAST ONE IS FOR FINAL LINEAR
    self.linears = clones(nn.Linear(d_model, d_model), 4)
    self.attn = None
    self.dropout = nn.Dropout(p=dropout)

  def forward(self, query, key, value, mask=None):
    if mask is not None:
      #apply the same mask to all the heads
      mask = mask.unsqueeze(1)
    nbatches = query.size(0)

    #now we do the linear projections from d_model => d_k
    #the projections for Q, K, V are all the same dimension

    #TODO: map out dimensions of each Q, K, V transform here

    ziplist = list(zip(self.linears, (query, key, value)))
    zipshapes = [(x[0], x[1].size()) for x in ziplist]


    query, key, value = [
        #apply linear projections
        lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1,2)
        for lin, x in zip(self.linears, (query, key, value))
    ]

    #apply attention on projections
    x, self.attn = attention(
        query, key, value, mask=mask, dropout=self.dropout
    )

    #Concat with a view and apply a linear layer
    x = (
        x.transpose(1,2)
        .contiguous()
        .view(nbatches, -1, self.h*self.d_k)
    )
    del query
    del key
    del value
    return self.linears[-1](x)

Note that in the **Attention is All You Need** paper, $h=8$ heads (parallel layers) are used. And in this case, $d_q=d_k=d_v=d_{\text{model}}/h = 64$.

**Section 3.2.3**:

#### Encoder Self-Attention:
In the encoder self-attention layer the queries, keys, and values all come from the prior layer's output. Each position can attend to every position in the prior layer.

We see this in the code for the `EncoderLayer` class when we use

    x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))

in the forward pass. Here, the mask will be `None` since no masking is needed.

#### Decoder Self-Attention:
Decoder self attention is similar to encoder attention except we employ the mask seen in `subsequent_mask` to mask out subsequent positions to prevent backward information flow in the output. We see how this essentially turns the illegal connections to `-inf` in the `attention` function with the line `scores = scores.masked_fill(mask == 0, -1e9)`. We see this in the `DecoderLayer` code forward pass in this line:

    x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))

#### Encoder-Decoder Attention:
In this case the queries come from the previous decoder layer while the keys and values come from the `memory` generated by the `Encoder` output. This is identical to *Seq2Seq* models' encoder-decoder attention. This is seen in the line immediately following the one where decoder self-attention is implemented in `DecoderLayer`:

    x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))

###Position-Wise FeedForward
**Section 3.3** outlines how the Position-wise Feed-Forward networks we use in the Transformer are defined.

The basic function defined is a composition of two linear layers with a ReLU activation between them:

$$\text{FFN}(x)=\text{max}(0,xW_1+b_1)W_2+b_2$$

In the paper $d_{\text{model}}/8=64$ so $d_{\text{model}} = 512$. We are also given the inner-layer dimension to be: $d_{ff}=2048=d_{\text{model}}*4$

In [137]:
class PositionwiseFeedForward(nn.Module):
  def __init__(self, d_model, d_ff, dropout=0.1):
    super(PositionwiseFeedForward, self).__init__()
    self.w_1 = nn.Linear(d_model, d_ff)
    self.w_2 = nn.Linear(d_ff, d_model)
    self.dropout = nn.Dropout(dropout)
    self.relu = nn.ReLU()

  def forward(self, x):
    return self.w_2(self.dropout(self.relu(self.w_1(x))))

**Section 3.4**:

###Embeddings and Positional Encoding


####Embeddings

We need to embed the input and output tokens into $d_{\text{model}}$-dimensional vectors. We use learned embeddings to do this. We share the weight matrix between the pre-softmax linear transform in both input and output embedding layers.

In [138]:
class Embeddings(nn.Module):
  def __init__(self, d_model, vocab):
    super(Embeddings, self).__init__()
    self.lut = nn.Embedding(vocab, d_model)
    self.d_model = d_model

  def forward(self, x):
    return self.lut(x)*math.sqrt(self.d_model)

`self.lut` above uses a linear transform to use a lookup table embedding. We multiply the pre-softmax linear transformation output with $\sqrt{d_{\text{model}}}$.

**Section 3.4**:

####Positional Encoding
We need a way to encode the relative or absolute positions of the words (tokens) in the sequence. We don't use any recurrence or convolution so we need to add information about token position in another way.

We use sinusoids scaled based on the dimension and position to differentiate positional data in the embedding vectors.

The exact details will become clear in the implementation below.

In [139]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, dropout, max_len=5000):
    super(PositionalEncoding, self).__init__()
    self.dropout = nn.Dropout(p=dropout)

    #max_len is max length of sequences allowed.
    pe = torch.zeros(max_len, d_model)
    position = torch.arange(0, max_len).unsqueeze(1)
    div_term = torch.exp(
        torch.arange(0, d_model, 2)*-(math.log(10000.0)/d_model)
    )
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    pe = pe.unsqueeze(0)

    #register_buffer is used because it doesn't
    #create a parameter. This means optimizer will not alter it
    self.register_buffer("pe", pe)

  def forward(self, x):
    x = x + self.pe[:, : x.size(1)].requires_grad_(False)
    return self.dropout(x)

In [140]:
#to get a better idea of how the positional encoding
#sinewaves look, we can do some visualization
def example_positional():
    pe = PositionalEncoding(20, 0)
    y = pe.forward(torch.zeros(1, 100, 20))

    data = pd.concat(
        [
            pd.DataFrame(
                {
                    "embedding": y[0, :, dim],
                    "dimension": dim,
                    "position": list(range(100)),
                }
            )
            for dim in [4, 5, 6, 7]
        ]
    )

    return (
        alt.Chart(data)
        .mark_line()
        .properties(width=800)
        .encode(x="position", y="embedding", color="dimension:N")
        .interactive()
    )


show_example(example_positional)

alt.Chart(...)

###The TRANSFORMER


We will now define a function that puts everything together to make a transformer model. All of the defaults for hyperparameters are identical to the hyperparameters used in the **Attention is All You Need** paper and have been mentioned in my descriptions earlier in this notebook.

In [141]:
def make_transformer(
    src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1
    ):
  c = copy.deepcopy
  attn = MultiHeadAttention(h, d_model)
  ff = PositionwiseFeedForward(d_model, d_ff, dropout)
  position = PositionalEncoding(d_model, dropout)
  transformer = EncoderDecoder(
      Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
      Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
      nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
      nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
      Generator(d_model, tgt_vocab),
  )
  # This is taken from The Annotated Transformer
  # Not sure why parameters are initialized in this specific manner
  for p in transformer.parameters():
      if p.dim() > 1:
          nn.init.xavier_uniform_(p)
  return transformer


##Model Training

###Inference Test

First let's test our model out on some dummy data of small size.

In [142]:
def inference_test():
  testformer = make_transformer(11, 11, N=2, h=8).to(DEVICE)
  testformer.eval()
  src = torch.LongTensor([[1,2,3,4,5,6,7,8,9,10]]).to(DEVICE)
  src_mask = torch.ones(1, 1, 10).to(DEVICE)

  memory = testformer.encode(src, src_mask)
  ys = torch.zeros(1, 1).type_as(src)

  for i in range(9):
    out = testformer.decode(
        memory=memory, src_mask=src_mask, tgt=ys, tgt_mask=subsequent_mask(ys.size(1)).type_as(src.data)
    )
    prob = testformer.generator(out[:, -1])
    _, next_word = torch.max(prob, dim=1)
    next_word = next_word.data[0]
    ys = torch.cat(
        [ys, torch.empty(1,1).type_as(src.data).fill_(next_word)], dim =1
    )

  print(f"Example Untrained Model Prediction: {ys}")



In [143]:
def run_tests():
  for _ in range(10):
    inference_test()

show_example(run_tests)

Example Untrained Model Prediction: tensor([[ 0,  8,  4, 10,  6,  0,  5,  0,  5,  0]], device='cuda:0')
Example Untrained Model Prediction: tensor([[0, 4, 9, 7, 2, 2, 2, 2, 2, 2]], device='cuda:0')
Example Untrained Model Prediction: tensor([[0, 5, 6, 6, 6, 6, 6, 6, 6, 6]], device='cuda:0')
Example Untrained Model Prediction: tensor([[0, 1, 1, 1, 8, 8, 8, 8, 8, 8]], device='cuda:0')
Example Untrained Model Prediction: tensor([[0, 1, 2, 9, 2, 9, 8, 8, 8, 8]], device='cuda:0')
Example Untrained Model Prediction: tensor([[ 0, 10,  6,  8,  5,  8,  8,  8,  8,  8]], device='cuda:0')
Example Untrained Model Prediction: tensor([[0, 7, 4, 3, 0, 1, 0, 7, 0, 7]], device='cuda:0')
Example Untrained Model Prediction: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')
Example Untrained Model Prediction: tensor([[0, 6, 6, 6, 6, 6, 6, 6, 3, 4]], device='cuda:0')
Example Untrained Model Prediction: tensor([[0, 3, 3, 3, 3, 3, 8, 5, 8, 5]], device='cuda:0')


###Training

We will make batch objects to train the model. This is also important when we want to parallelize training even though that is outside the scope of this particular exrecise of mine.

In [144]:
class Batch:
  #Object to hold a batch of training data with mask.
  def __init__(self, src, tgt=None, pad=2):
    #Pad determines which token is <blank>
    self.src = src
    self.src_mask = (src != pad).unsqueeze(-2)
    if tgt is not None:
      self.tgt = tgt[:, :-1] #TODO: what?
      self.tgt_y = tgt[:, 1:] #TODO: what?
      self.tgt_mask = self.make_std_mask(self.tgt, pad)
      # print(f"{self.tgt.device=}")
      # print(f"{self.tgt_y.device=}")
      # print(f"{self.tgt_mask.device=}")
      self.ntokens = (self.tgt_y != pad).data.sum() #just a count of non pad tokens
      # print(f"{self.ntokens.device=}")

  @staticmethod
  def make_std_mask(tgt, pad):
    #Mask to hide padding and future words in target sentences.
    tgt_mask = (tgt != pad).unsqueeze(-2)
    tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data)
    return tgt_mask


In [145]:
class TrainState:
  #Track number of steps, examples, and tokens processed
  step: int = 0 #steps in current epoch
  accum_step: int = 0 #gradient accumulation steps
  samples: int = 0 #total num of examples used
  tokens: int = 0 #total num of tokens processed

def run_epoch(
    data_iter,
    model,
    loss_compute,
    optimizer,
    scheduler,
    mode="train",
    accum_iter=1,
    train_state=TrainState(),
    ):
  #single epoch training
  start = time.time()
  total_tokens = 0
  total_loss = 0
  tokens = 0
  n_accum = 0
  for i, batch in enumerate(data_iter):
    #forward pass
    # model.train()
    out = model.forward(
        batch.src, batch.tgt, batch.src_mask, batch.tgt_mask
        )
    #compute loss on prediction
    loss, loss_node = loss_compute(out, batch.tgt_y, batch.ntokens)
    if mode == "train" or mode == "train+log":
      #backpropagate loss
      loss_node.backward()
      #record training data in train_state
      train_state.step += 1
      train_state.samples += batch.src.shape[0]
      train_state.tokens += batch.ntokens
      if i % accum_iter == 0:
        #accumulate gradient and update weights
        optimizer.step()
        #zero out gradient
        optimizer.zero_grad(set_to_none=True)#why set to none?
        n_accum += 1
        train_state.accum_step +=1
      scheduler.step()
    total_loss += loss
    total_tokens += batch.ntokens
    tokens += batch.ntokens
    if i % 40 == 1 and (mode == "train" or mode == "train+log"):
      lr = optimizer.param_groups[0]["lr"]
      elapsed = time.time()-start
      start = time.time()
      tokens = 0
    del loss
    del loss_node
  return total_loss/total_tokens, train_state

In [146]:
#Learning Rate Schedule
def learning_rate(step, model_size, factor, warmup):
  if step == 0:
    step = 1
  return factor * (model_size**(-0.5) * min(step**(-0.5), step*warmup**(-0.5)))

###Regularization

The paper states that Label Smoothing was employed during training. I am not totally sure of the details of this.

Label Smoothing is implemented here using the [KL Divergence Loss](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) function.

In [147]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."

    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction="sum")
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None

    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, true_dist.clone().detach())

In [148]:
def example_label_smoothing():
    crit = LabelSmoothing(5, 0, 0.4)
    predict = torch.FloatTensor(
        [
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
        ]
    )
    crit(x=predict.log(), target=torch.LongTensor([2, 1, 0, 3, 3]))
    LS_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "target distribution": crit.true_dist[x, y].flatten(),
                    "columns": y,
                    "rows": x,
                }
            )
            for y in range(5)
            for x in range(5)
        ]
    )
    return None

    # return (
    #     alt.Chart(LS_data)
    #     .mark_rect(color="Blue", opacity=1)
    #     .properties(height=200, width=200)
    #     .encode(
    #         alt.X("columns:O", title=None),
    #         alt.Y("rows:O", title=None),
    #         alt.Color(
    #             "target distribution:Q", scale=alt.Scale(scheme="viridis")
    #         ),
    #     )
    #     .interactive()
    # )


show_example(example_label_smoothing)

###Synthetic Data Training Example

We will generate synthetic data of a small vocabulary to train our model to copy data. (Maybe reverse it?)

In [190]:
def data_gen(V, batch_size, nbatches, start_symbol = 1):
  #copy-data task
  for i in range(nbatches):
    data = torch.randint(1, V, size=(batch_size, 10))
    data[:,0] = start_symbol
    src = data.requires_grad_(False).clone().detach().to(DEVICE)
    tgt = data.requires_grad_(False).clone().detach().to(DEVICE)
    yield Batch(src, tgt, 0)

def reverse_data_gen(V, batch_size, nbatches, start_symbol = 0):
  for i in range(nbatches):
    data = torch.randint(1, V, size=(batch_size, 10))
    src = data.requires_grad_(False).clone().detach().to(DEVICE)
    src[:,0] = start_symbol
    data = torch.flip(data, dim=-1)
    tgt = data.requires_grad_(False).clone().detach().to(DEVICE)
    tgt[:,0] = start_symbol
    yield Batch(src, tgt, 0)

####Loss Computation

In [191]:
class SimpleLossCompute:
  def __init__(self, generator, criterion):
    self.generator = generator
    self.criterion = criterion

  def __call__(self, x, y, norm):
    x = self.generator(x)
    sloss = (
        self.criterion(
            x.contiguous().view(-1, x.size(-1)),
            y.contiguous().view(-1)
        ) / norm
    )
    return sloss.data * norm, sloss

####Greedy Decoding

This code predicts a translation using greedy decoding for simplicity.

In [192]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    #note this is similar to the decode loop in our
    #inference example
    # print("+++++GREEDY_DECODE+++++")
    memory = model.encode(src, src_mask)
    ys = torch.zeros(1, 1).fill_(start_symbol).type_as(src.data)
    # print(f"=====MEMORY:{memory.shape}======")
    # print(f"{memory[:,2,:20]=}")
    for i in range(max_len - 1):
      # print(f"POSITION:{i}")
      # print(f"{ys=}")
      out = model.decode(
          memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
      )
      prob = model.generator(out[:, -1])
      _, next_word = torch.max(prob, dim=1)
      next_word = next_word.data[0]
      ys = torch.cat(
          [ys, torch.zeros(1, 1).type_as(src.data).fill_(next_word)], dim=1
      )
    return ys


####Train Simple Copy Task

In [193]:
# Train the simple copy task.
trained_model = None

def example_simple_model():
  torch.manual_seed(99)
  V = 11
  criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.1)
  model = make_transformer(V, V, N=2).to(DEVICE)

  optimizer = torch.optim.Adam(
      model.parameters(), lr=0.5, betas=(0.9, 0.98), eps=1e-9
  )
  lr_scheduler = torch.optim.lr_scheduler.LambdaLR(
      optimizer=optimizer,
      lr_lambda=lambda step: learning_rate(
          step, model_size=model.src_embed[0].d_model, factor=1.0, warmup=800
      ),
  )

  batch_size = 80
  nepochs = 300
  nbatches = 20
  for epoch in range(nepochs):
    if epoch%10==0:
      print(f"=====EPOCH:{epoch}=====")
    model.train()
    run_epoch(
        data_gen(V, batch_size, nbatches),
        model,
        SimpleLossCompute(model.generator, criterion),
        optimizer,
        lr_scheduler,
        mode="train",
    )
    model.eval()
    loss_value = run_epoch(
        data_gen(V, batch_size, 5),
        model,
        SimpleLossCompute(model.generator, criterion),
        DummyOptimizer(),
        DummyScheduler(),
        mode="eval",
    )[0]
    if epoch%10==0:
      print(f"{loss_value=}")

  model.eval()
  src = torch.LongTensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]).to(DEVICE)
  max_len = src.shape[1]
  src_mask = torch.ones(1, 1, max_len).to(DEVICE)
  print(f"{src=}|{greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0)=}")
  src = torch.LongTensor([[0, 1, 3, 3, 4, 6, 6, 7, 8, 9]]).to(DEVICE)
  print(f"{src=}|{greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0)=}")
  src = torch.LongTensor([[0, 8, 7, 6, 5, 4, 6, 7, 8, 9]]).to(DEVICE)
  print(f"{src=}|{greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0)=}")
  src = torch.LongTensor([[0, 9, 8, 7, 6, 5, 4, 3, 2, 1]]).to(DEVICE)
  print(f"{src=}|{greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0)=}")
  src = torch.LongTensor([[0, 4, 4, 4, 4, 4, 4, 4, 4, 4]]).to(DEVICE)
  print(f"{src=}|{greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0)=}")

  trained_model = model

execute_example(example_simple_model)

=====EPOCH:0=====
loss_value=tensor(1.5076, device='cuda:0')
=====EPOCH:10=====
loss_value=tensor(1.3653, device='cuda:0')
=====EPOCH:20=====
loss_value=tensor(1.2910, device='cuda:0')
=====EPOCH:30=====
loss_value=tensor(1.2180, device='cuda:0')
=====EPOCH:40=====
loss_value=tensor(1.1055, device='cuda:0')
=====EPOCH:50=====
loss_value=tensor(1.0505, device='cuda:0')
=====EPOCH:60=====
loss_value=tensor(1.0112, device='cuda:0')
=====EPOCH:70=====
loss_value=tensor(0.9838, device='cuda:0')
=====EPOCH:80=====
loss_value=tensor(0.9447, device='cuda:0')
=====EPOCH:90=====
loss_value=tensor(0.8516, device='cuda:0')
=====EPOCH:100=====
loss_value=tensor(0.7837, device='cuda:0')
=====EPOCH:110=====
loss_value=tensor(0.7612, device='cuda:0')
=====EPOCH:120=====
loss_value=tensor(0.7508, device='cuda:0')
=====EPOCH:130=====
loss_value=tensor(0.6752, device='cuda:0')
=====EPOCH:140=====
loss_value=tensor(0.6500, device='cuda:0')
=====EPOCH:150=====
loss_value=tensor(0.5894, device='cuda:0')
===

####Results
As we can see above, the model takes some time but it does learn reasonably well how to copy text.

It still messes up sometimes but the model shows good performance. Let's try some random inputs.

Below you see the comparisons of some of the runs when the data generation was changed up.

In [183]:
##OUTPUT WITH DATAGEN START NUM = 1
# =====EPOCH:0=====
# loss_value=tensor(1.5076, device='cuda:0')
# =====EPOCH:10=====
# loss_value=tensor(1.3653, device='cuda:0')
# =====EPOCH:20=====
# loss_value=tensor(1.2910, device='cuda:0')
# =====EPOCH:30=====
# loss_value=tensor(1.2180, device='cuda:0')
# =====EPOCH:40=====
# loss_value=tensor(1.1055, device='cuda:0')
# =====EPOCH:50=====
# loss_value=tensor(1.0505, device='cuda:0')
# =====EPOCH:60=====
# loss_value=tensor(1.0112, device='cuda:0')
# =====EPOCH:70=====
# loss_value=tensor(0.9838, device='cuda:0')
# =====EPOCH:80=====
# loss_value=tensor(0.9447, device='cuda:0')
# =====EPOCH:90=====
# loss_value=tensor(0.8516, device='cuda:0')
# =====EPOCH:100=====
# loss_value=tensor(0.7837, device='cuda:0')
# =====EPOCH:110=====
# loss_value=tensor(0.7612, device='cuda:0')
# =====EPOCH:120=====
# loss_value=tensor(0.7508, device='cuda:0')
# =====EPOCH:130=====
# loss_value=tensor(0.6752, device='cuda:0')
# =====EPOCH:140=====
# loss_value=tensor(0.6500, device='cuda:0')
# =====EPOCH:150=====
# loss_value=tensor(0.5894, device='cuda:0')
# =====EPOCH:160=====
# loss_value=tensor(0.5709, device='cuda:0')
# =====EPOCH:170=====
# loss_value=tensor(0.5424, device='cuda:0')
# =====EPOCH:180=====
# loss_value=tensor(0.5243, device='cuda:0')
# =====EPOCH:190=====
# loss_value=tensor(0.5240, device='cuda:0')
# src=tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], device='cuda:0')|greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0)=tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], device='cuda:0')
# src=tensor([[0, 1, 3, 3, 4, 6, 6, 7, 8, 9]], device='cuda:0')|greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0)=tensor([[0, 1, 3, 4, 4, 6, 9, 7, 8, 9]], device='cuda:0')
# src=tensor([[0, 8, 7, 6, 5, 4, 6, 7, 8, 9]], device='cuda:0')|greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0)=tensor([[0, 7, 1, 8, 6, 4, 6, 0, 8, 9]], device='cuda:0')
# src=tensor([[0, 9, 8, 7, 6, 5, 4, 3, 2, 1]], device='cuda:0')|greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0)=tensor([[0, 1, 8, 7, 6, 5, 4, 3, 2, 1]], device='cuda:0')
# src=tensor([[0, 4, 4, 4, 4, 4, 4, 4, 4, 4]], device='cuda:0')|greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0)=tensor([[0, 1, 8, 5, 4, 4, 4, 4, 4, 4]], device='cuda:0')



tensor([[ 3,  5,  6,  2,  2,  5,  1,  3,  5,  5],
        [ 3,  7,  4,  2, 10,  4,  1,  8,  4,  7]])
tensor([[ 1,  5,  6,  2,  2,  5,  1,  3,  5,  5],
        [ 1,  7,  4,  2, 10,  4,  1,  8,  4,  7]])


AttributeError: ignored

In [154]:
##OUTPUT WITH DATAGEN STARTNUM = 0
# =====EPOCH:0=====
# loss_value=tensor(1.4342, device='cuda:0')
# =====EPOCH:10=====
# loss_value=tensor(1.2491, device='cuda:0')
# =====EPOCH:20=====
# loss_value=tensor(1.1966, device='cuda:0')
# =====EPOCH:30=====
# loss_value=tensor(1.1475, device='cuda:0')
# =====EPOCH:40=====
# loss_value=tensor(1.0529, device='cuda:0')
# =====EPOCH:50=====
# loss_value=tensor(1.0130, device='cuda:0')
# =====EPOCH:60=====
# loss_value=tensor(0.9728, device='cuda:0')
# =====EPOCH:70=====
# loss_value=tensor(0.9412, device='cuda:0')
# =====EPOCH:80=====
# loss_value=tensor(0.9101, device='cuda:0')
# =====EPOCH:90=====
# loss_value=tensor(0.8762, device='cuda:0')
# =====EPOCH:100=====
# loss_value=tensor(0.8318, device='cuda:0')
# =====EPOCH:110=====
# loss_value=tensor(0.8321, device='cuda:0')
# =====EPOCH:120=====
# loss_value=tensor(0.8194, device='cuda:0')
# =====EPOCH:130=====
# loss_value=tensor(0.7708, device='cuda:0')
# =====EPOCH:140=====
# loss_value=tensor(0.7733, device='cuda:0')
# =====EPOCH:150=====
# loss_value=tensor(0.7235, device='cuda:0')
# =====EPOCH:160=====
# loss_value=tensor(0.7316, device='cuda:0')
# =====EPOCH:170=====
# loss_value=tensor(0.6779, device='cuda:0')
# =====EPOCH:180=====
# loss_value=tensor(0.6648, device='cuda:0')
# =====EPOCH:190=====
# loss_value=tensor(0.6774, device='cuda:0')
# src=tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], device='cuda:0')|greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0)=tensor([[0, 0, 0, 0, 0, 4, 5, 7, 8, 9]], device='cuda:0')
# src=tensor([[0, 1, 3, 3, 4, 6, 6, 7, 8, 9]], device='cuda:0')|greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0)=tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')
# src=tensor([[0, 8, 7, 6, 5, 4, 6, 7, 8, 9]], device='cuda:0')|greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0)=tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')
# src=tensor([[0, 9, 8, 7, 6, 5, 4, 3, 2, 1]], device='cuda:0')|greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0)=tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')
# src=tensor([[0, 4, 4, 4, 4, 4, 4, 4, 4, 4]], device='cuda:0')|greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0)=tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

